In [1]:
import numpy as np
import novosparc
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import scanpy as sc
import altair as alt
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans

import random

random.seed(33)

/usr/local/opt/pyenv/versions/3.7.5/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


# Reconstructing Osteosarcoma

Here we optimize reconstruction for the attempt [MERFISH osteosarcoma](https://www.pnas.org/content/116/39/19490) spatial expression. Specifically we examine the effect of each assumption and their integration:

1. Smoothness assumption - we notice expression microenvironments thus we construct two tissues, one accounting for all physical distances and one just for short distances.
2. Linear assumption - how many markers are required for successful reconstruction using only markers?
3. Integration - combining the two terms, does the smoothness assumption benefit existing markers?

In [2]:
# params

n_neighbors = 5
dataname = 'osteosarcoma'
coords_cols=['xcoord', 'ycoord']
gois = ['PKM', 'RPL36A','FGF18', 'SMAD3']
ngois = len(gois)

batch = 1
data_path = 'novosparc/datasets/%s/dge_%d.txt' % (dataname, batch)
target_space_path = 'novosparc/datasets/%s/geometry_%d.txt' % (dataname, batch)

In [3]:
# plotting 

max_pts = 5000
pt_size = 50
    
labelFontSize=15
labelFontSizeb=17
titleFontSize=20
fontSize=20

def to_paper(pl):
    pl = pl.configure_axis(labelFontSize=labelFontSize, titleFontWeight='normal', titleFontSize=titleFontSize)
    pl = pl.configure_title(fontSize=fontSize)
    pl = pl.configure_legend(titleFontSize=labelFontSize,labelFontSize=labelFontSize) 
    return pl 
                             
def to_zscore(df_goi):
    return (df_goi - df_goi.mean(0)).div(df_goi.std(0))

def norm(df_goi):
    return (df_goi - df_goi.min(0)).div(df_goi.max(0) - df_goi.min(0))


def plot_df_sp(df_sp, tit='', gois=gois):
    """
    Plots spatial expression.
    : param df_sp: dataframe containing locations and expression for genes-of-interest (gois)
    """
    cdf_sp = df_sp.copy()
    vals = cdf_sp['true_pred'].unique()
    for v in vals:
        cdf_sp.loc[cdf_sp['true_pred'] == v, gois] = to_zscore(cdf_sp.loc[cdf_sp['true_pred'] == v, gois])

    scale_axis = alt.Scale(nice=False)
    axis_rmgrid = alt.Axis(grid=False, values=[-1000, 0, 1000])
    header_size = alt.Header(labelFontSize=labelFontSizeb, labelFontWeight='bold')
    scale_color=alt.Scale(domain=(-2, 0, 2),range=[ 'blue', 'white', 'red'])
    
    pl = alt.Chart(cdf_sp, title=tit).mark_circle(size=pt_size).transform_fold(gois, ['goi', 'Expression'])
    pl = pl.encode( x=alt.X('xcoord:Q', scale=scale_axis, axis=axis_rmgrid), 
                    y=alt.Y('ycoord:Q', scale=scale_axis, axis=axis_rmgrid), 
                    row=alt.Row('goi:N', sort=None, title=None, header=header_size),
                    column=alt.Column('true_pred:N', title=None,  header=header_size),
                    color=alt.Color('Expression:Q', scale=scale_color))
    pl = pl.configure_axis(grid=False).configure_view(strokeOpacity=0)
    to_paper(pl).display()


In [4]:
# quantitative evaluation of reconstruction

def comp_corr(df_dge1, df_dge2):
    """
    Computes the correlation of expression for each gene between true and predicted spatial expression.
    """
    corrs = {}
    for g in df_dge1.columns:
        corrs[g] = np.corrcoef(df_dge1[g], df_dge2[g])[0,1]
    return corrs

def get_corrs_median(gene_corrs):
    return np.median([v for _,v in gene_corrs.items()])

In [5]:
# read data
dataset = novosparc.io.load_data(data_path)
sc.pp.normalize_per_cell(dataset)
sc.pp.log1p(dataset)

locations = novosparc.io.load_target_space(target_space_path, coords_cols=coords_cols)

df_dge = dataset.to_df() # cells x genes
df_locs = pd.DataFrame(locations, columns=coords_cols)
df_dge.index = df_locs.index
ncells,ngenes = df_dge.shape

df_sp_true = pd.concat((df_dge[gois], df_locs), 1)
df_sp_true['true_pred'] = 'True'

plot_df_sp(df_sp_true, tit='True Spatial Expression')

alt.Chart(...)

In [6]:
# to remove noise and ease computation of cell-cell distances, using pca representation

n_comps = 30
sc.pp.pca(dataset, n_comps=n_comps)
df_dge_pca = pd.DataFrame(dataset.obsm['X_pca'], columns=['pc_%d' % i for i in np.arange(n_comps)])
dataset_pca = sc.AnnData(df_dge_pca)


Transforming to str index.


In [7]:
# showing for pca representation, can also run for original dataset
df_dge_chosen = df_dge_pca

In [8]:
# to show microenvironments, using simple clustering of expression representation

# kmeans
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df_dge_pca)
km_cluster = kmeans.labels_

df_kmeans = df_locs.copy()
df_kmeans['Kmean Cluster'] = km_cluster

tit='MERFISH Osteosarcoma Microenvironments'

scale_tight = alt.Scale(nice=False)
axis_rmgrid = alt.Axis(grid=False, values=[-1000, 0, 1000])
header_size = alt.Header(labelFontSize=labelFontSizeb, labelFontWeight='bold')

pl = alt.Chart(df_kmeans, title=tit).mark_circle(size=pt_size).encode(x=alt.X('xcoord:Q', scale=scale_tight, axis=axis_rmgrid), 
                                                                 y=alt.Y('ycoord:Q', scale=scale_tight, axis=axis_rmgrid), 
                                                                 color='Kmean Cluster:N')
to_paper(pl)

alt.Chart(...)

## Only Smoothness Assumption and Microenvironments

- So far we see there are expression niches in the tissue (kmeans of pca representation).

- Having the expression per location here (the desired output), we can test the smoothness assumption assuming preservation of physical and expression distances. 

- Looking at the distance and cost comparisons as well as the microenvironments above, we see the assumption holds for small physical distances. 

- Therefore, we test here two options of reconstruction, accounting for all physical distances, or prior knowing  expression similarity exists in niches and thus we threshold the location cost matrix. 

In [10]:
# look at exp dists - loc dists

dist_locs = pdist(locations)
dist_exp = pdist(df_dge_chosen)

df_dists = pd.DataFrame({'Location Dist': dist_locs,
                        'Expression Dist': dist_exp})

tit = 'Expression Dist vs Location Dist (Corr: %.02f)' % np.corrcoef(dist_locs, dist_exp) [0,1]

base = alt.Chart(alt.sample(df_dists, n=max_pts), title=tit)

bin_x = alt.X('Location Dist:Q', bin=True)
band = base.mark_errorband(extent="iqr").encode(x=bin_x, y='Expression Dist:Q', 
                                   color=alt.value('blue'), opacity=alt.value(0.5))
box = base.mark_boxplot().encode(x=bin_x, y='Expression Dist:Q', 
                                   color=alt.value('blue'), opacity=alt.value(0.5))

means = base.mark_line().encode(x=bin_x, y='mean(Expression Dist)', color=alt.value('black'))

to_paper(box + means | band + means) 

alt.HConcatChart(...)

In [11]:
# calculate cost matrices for OT
n_neighbors = 2
cost_exp, cost_locs = novosparc.rc.setup_for_OT_reconstruction(df_dge_chosen.values, 
                                                               locations,
                                                               num_neighbors_source = n_neighbors,
                                                               num_neighbors_target = n_neighbors)
idx = np.tril_indices_from(cost_locs, k=1)
df_costs = pd.DataFrame({'Location Cost': cost_locs[idx],
                        'Expression Cost': cost_exp[idx]})

tit = 'Expression Cost vs Location Cost (Corr: %.02f)' % np.corrcoef(cost_locs[idx], cost_exp[idx])[0,1]
base = alt.Chart(alt.sample(df_costs, n=max_pts), title=tit)

bin_x = alt.X('Location Cost', bin=True)

band = base.mark_errorband(extent="iqr").encode(x=bin_x, y='Expression Cost', color=alt.value('blue'), opacity=alt.value(0.5))
box = base.mark_boxplot().encode(x=bin_x, y='Expression Cost', color=alt.value('blue'), opacity=alt.value(0.5))
means = base.mark_line().encode(x=bin_x, y='mean(Expression Cost)', color=alt.value('black'))

to_paper(box + means | band + means)

Setting up for reconstruction ... done ( 0.03 seconds )


alt.HConcatChart(...)

In [12]:
# smoothness, regular assumption
tissue_all = novosparc.cm.Tissue(dataset=dataset, locations=locations)
tissue_all.setup_smooth_costs(dge_rep=df_dge_chosen, 
                          num_neighbors_s=n_neighbors, 
                          num_neighbors_t=n_neighbors, 
                          verbose=False)


# smoothness, thresholding small physical distances
tissue_spd = novosparc.cm.Tissue(dataset=dataset, locations=locations) # small physical distances
tissue_spd.setup_smooth_costs(dge_rep=df_dge_chosen, 
                          num_neighbors_s=n_neighbors, 
                          num_neighbors_t=n_neighbors, 
                          verbose=False)
dist_thr = -0.3
cost_locs = tissue_spd.costs['locations']
idx_utri = np.triu_indices_from(cost_locs, k=1)

n_locs_bellow_thr = np.sum((cost_locs < dist_thr)[idx_utri])  
n_locs = np.sum(np.ones_like(cost_locs[idx_utri]))
print('Num location pairs with cost < %.02f: %d / %d (%.02f)' % (dist_thr, 
                                                                 n_locs_bellow_thr, 
                                                                 n_locs, 
                                                                 n_locs_bellow_thr/n_locs))

cost_locs_prev = np.copy(cost_locs)
cost_locs[cost_locs >= dist_thr] = dist_thr


tissues = {'All Physical Distances': tissue_all, 
          'Small Physical Distances Only': tissue_spd}

Num location pairs with cost < -0.30: 1006 / 207690 (0.00)


### Denovo Reconstruction

In [13]:
# denovo with chosen representation, title (median gene correlation)
alpha_linear = 0

for tissue_desc, tissue in tissues.items():
    tissue.reconstruct(alpha_linear=alpha_linear, epsilon=5e-3, verbose=False)

    df_sdge = (df_dge.T @ tissue.gw).T
    df_sp_pred = pd.concat((df_sdge[gois].astype('float32'), pd.DataFrame(locations, columns=coords_cols)), 1)
    df_sp_pred['true_pred'] = 'Pred'

    gene_corrs = comp_corr(df_sdge, df_dge)
    tit = 'Denovo %s (%.02f)' % (tissue_desc, get_corrs_median(gene_corrs))

    plot_df_sp(pd.concat((df_sp_true, df_sp_pred),0), tit=tit)


alt.Chart(...)

alt.Chart(...)

In [14]:
# # looking at coords as genes

# coord_genes = ['gene_xcoord', 'gene_ycoord']
# df_dge_locs = pd.concat((df_dge, df_locs), 1)
# df_dge_locs = df_dge_locs.rename(columns={'xcoord':'gene_xcoord', 'ycoord':'gene_ycoord'})
# df_sdge_locs = (df_dge_locs.T @ tissue.gw).T

# df_sp_true_locs = pd.concat((df_dge_locs[coord_genes].astype('float32'), df_locs), 1)
# df_sp_true_locs['true_pred'] = 'True'


# df_sp_pred_locs = pd.concat((df_sdge_locs[coord_genes].astype('float32'), df_locs), 1)
# df_sp_pred_locs['true_pred'] = 'Pred'

# tit='coords'
# plot_df_sp(pd.concat((df_sp_true_locs, df_sp_pred_locs),0), tit=tit, gois=coord_genes)

## Only Linear Assumption and Number of Markers

- looking at a known marker

- rnadomly varying the markers selected
Here we are going to reconstruct only with markers, first using a single marker we know and then randomly choosing a marker. In order to compare and evaluate the results we are going to look at the median (over all genes) of the correlation between a gene's true and predicted expression. Note that this is a naive evaluatino just for demonstration but more appropriate evaluations such as spatial correlation (cross Moran's I) are available. 

## Noise

As often the markers are not exact, we also attempt reconstruction where there is some noise added to the markers

In [15]:
eps = 0.1
df_dge_noise = df_dge + eps * np.random.normal(loc=0, scale=1, size=(ncells, ngenes))
df_sp_noise = pd.concat((df_dge_noise[gois], df_locs), 1)
df_sp_noise['true_pred'] = 'Noisy'

# showing high correlation and looks the same but not exact
gene_corrs = comp_corr(df_dge, df_dge_noise)
plot_df_sp(pd.concat((df_sp_true, df_sp_noise), 0), tit='True and Noisy Spatial Expression (%.02f)' % (get_corrs_median(gene_corrs)))

alt.Chart(...)

In [16]:
df_markers = {'same': df_dge,
             'noise_%.02f' % eps: df_dge_noise}

In [17]:
# params
idx = np.arange(ngenes)
genenames = df_dge.columns
alpha_linear = 1.0

In [18]:
# using PKM only
for df_marker_desc, df_marker in df_markers.items():
    marker_name = 'PKM'
    marker_idx = np.where(genenames == marker_name)[0]
    marker_names = [marker_name]

    tissue.setup_linear_cost(markers_to_use=marker_idx, 
                            insitu_matrix=df_marker[marker_names].values)

    tissue.reconstruct(alpha_linear=alpha_linear, epsilon=5e-3, verbose=False)

    df_sdge = (df_dge.T @ tissue.gw).T

    gene_corrs = comp_corr(df_dge, df_sdge)

    df_sp_pred = pd.concat((df_sdge[gois].astype('float32'), pd.DataFrame(locations, columns=coords_cols)), 1)
    df_sp_pred['true_pred'] = 'Pred'

    tit = '%s : Using %s Marker Only (%.02f)' % (df_marker_desc, marker_name, get_corrs_median(gene_corrs))
    plot_df_sp(pd.concat((df_sp_true, df_sp_pred),0), tit=tit, gois=[g for g in gois if g!= marker_name])

alt.Chart(...)

alt.Chart(...)

In [19]:
# tit = 'Normalized Expression using %s Marker Only (Median Correlation of all Genes: %.02f)' % (marker_name, np.median([v for _,v in gene_corrs.items()]))

# df_sp_pred = pd.concat((df_sdge[gois].astype('float32'), df_locs), 1)
# df_sp_pred['true_pred'] = 'Pred'


# df_sp_true_z = df_sp_true.copy()
# df_sp_true_z[gois] = norm(to_zscore(df_sp_true[gois]))

# df_sp_pred_z = df_sp_pred.copy()
# df_sp_pred_z[gois] = norm(to_zscore(df_sp_pred[gois]))

# plot_df_sp(pd.concat((df_sp_true_z, df_sp_pred_z),0), tit=tit)

In [20]:
# # # Evaluation of reconstruction with random selection of markers
# # alpha_linear = 1.0
# # n_repeats = 5
# # n_markers = [1,2,4,8]

# # results_markers = []


# # for i in np.arange(n_repeats):
# #     for n_marker in n_markers:
# #         np.random.shuffle(idx)
# #         marker_idx = idx[:n_marker]
# #         marker_names = genenames[marker_idx]
        
# #         tissue.setup_linear_cost(markers_to_use=marker_idx, 
# #                                 insitu_matrix=df_dge[marker_names].values)

# #         tissue.reconstruct(alpha_linear=alpha_linear, epsilon=5e-3, verbose=False)

# #         df_sdge = (df_dge.T @ tissue.gw).T
        
# #         gene_corrs = comp_corr(df_dge, df_sdge)
# #         median = get_corrs_median(gene_corrs)
        
# #         results_markers.append({'median gene corr': median,
# #                                'num markers': n_marker})
        
# df_results_markers = pd.DataFrame(results_markers)
# # df_results_markers['median gene corr'] = df_results_markers[genenames].median(1)

# tit = 'Reconstruction Only by Markers'
# pl_width = 400
# base = alt.Chart(df_results_markers, title=tit, width=pl_width)
# pl_markers = base.mark_boxplot().encode(
#     x=alt.X('num markers:N', axis=alt.Axis(labelAngle=0)),
#     y=alt.Y('median gene corr:Q'),
#     color='num markers:N')
# pl_mean_markers = base.mark_point().encode(
#     x=alt.X('num markers:N', axis=alt.Axis(labelAngle=0)),
#     y='mean(median gene corr)',
#     color=alt.value('black')
# )

# to_paper(pl_markers + pl_mean_markers)

## Integration of Smoothness and Linear

- testing smoothness assumption, all distances or just small physical distances
- 

{'All Physical Distances': <novosparc.common._tissue.Tissue at 0x132731810>,
 'Small Physical Distances Only': <novosparc.common._tissue.Tissue at 0x132731490>}

In [23]:
# Evaluation of reconstruction with random selection of markers

n_repeats = 5
n_markers = [1, 2, 4]
alpha_linears = [0.25, 0.5, 0.75, 1.0]


results_sl = [] # smooth linear

tissue_desc = 'All Physical Distances'
tissue = tissues[tissue_desc]


for n_marker in n_markers:
    for i in np.arange(n_repeats):
        np.random.shuffle(idx)
        marker_idx = idx[:n_marker]
        marker_names = genenames[marker_idx]

        for df_marker_desc, df_marker in df_markers.items():
            for tissue_desc, tissue in tissues.items():
                tissue.setup_linear_cost(markers_to_use=marker_idx, 
                                                    insitu_matrix=df_marker[marker_names].values) # only choice of markers varies

                for alpha_linear in alpha_linears:
                    tissue.reconstruct(alpha_linear=alpha_linear, epsilon=5e-3, verbose=False)

                    df_sdge = (df_dge.T @ tissue.gw).T

                    gene_corrs = comp_corr(df_dge, df_sdge)
                    median = get_corrs_median(gene_corrs)
                    results_sl.append({'median gene corr': median,
                                      'tissue desc': tissue_desc,
                                      'num markers': n_marker,
                                      'alpha linear': alpha_linear,
                                      'markers desc': df_marker_desc})


df_results_sl = pd.DataFrame(results_sl)

# tit = 'Integration of Smooth and Linear Assumptions'
# base = alt.Chart(df_results_sl[['alpha linear', 'median gene corr']], title=tit)

# scale_x = alt.Scale(domain=(-0.05,1.05))
# scale_y = alt.Scale(domain=(0,1))

# pl_linear = base.mark_boxplot().encode(
#     x=alt.X('alpha linear:Q', scale=scale_x),
#     y=alt.Y('median gene corr:Q', scale=scale_y),
#     row=alt.Row('num markers:N'),
#     color='tissue desc:N')
# pl_mean_linear = base.mark_point().encode(
#     x=alt.X('alpha linear:Q', scale=scale_x),
#     y=alt.Y('mean(median gene corr)', scale=scale_y),
#     row=alt.Row('num markers:N'),
#     color='tissue desc:N')

# to_paper(pl_linear + pl_mean_linear)

In [24]:
pl_width = 200
tit = 'Integration of Smooth and Linear Assumptions'
base = alt.Chart(df_results_sl, title=tit, width=pl_width)
pl_sl = base.mark_circle().encode(x='alpha linear:N',
                                              y=alt.Y('mean(median gene corr):Q', scale=alt.Scale(domain=[-0.1,1])),
                                              color='tissue desc:N',
                                              column='num markers:N',
                                 row='markers desc:N')
to_paper(pl_sl)

alt.Chart(...)

In [25]:
pl_width = 200
tit = 'Integration of Smooth and Linear Assumptions'
base = alt.Chart(df_results_sl, title=tit, width=pl_width)
pl_sl = base.mark_boxplot().encode(x='alpha linear:N',
                                              y=alt.Y('median gene corr:Q', scale=alt.Scale(domain=[-0.1,1])),
                                              color='tissue desc:N',
                                              column='num markers:N',
                                  row='markers desc:N')
to_paper(pl_sl)

alt.Chart(...)

In [26]:
desc = 'noise_pca_2nn'
# df_results_markers.to_csv('results_markers')
df_results_sl.to_csv('df_results_sl_%s' % desc)

In [29]:
df_results_sl_2nn = df_results_sl
df_results_sl_5nn = pd.read_csv('df_results_sl_noise_pca_5nn')

In [30]:
df_results_sl_2nn['nn'] = 2
df_results_sl_5nn['nn'] = 5

In [36]:
df = pd.concat((df_results_sl_2nn, df_results_sl_5nn))

pl_width = 200
tit = 'Integration of Smooth and Linear Assumptions'
base = alt.Chart(df[df['tissue desc'] == 'All Physical Distances'], title=tit, width=pl_width)
pl_sl = base.mark_boxplot().encode(x='alpha linear:N',
                                              y=alt.Y('mean(median gene corr):Q', scale=alt.Scale(domain=[-0.1,1])),
                                              color='nn:N',
                                              column='num markers:N',
                                  row='markers desc:N')
to_paper(pl_sl)

alt.Chart(...)